In [15]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Activation, RepeatVector, Dropout, Dense
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
import numpy as np
import os 

# Suppress the TensorFlow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [16]:
# gensim
import gensim
from gensim.models import KeyedVectors, Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.test.utils import datapath
from gensim.models.word2vec import  PathLineSentences, LineSentence
from gensim import utils
from time import time
from time import time
from os import listdir
from tqdm import tqdm
import os
import logging
import numpy as np

In [17]:
folder = 'train_data/training-monolingual.tokenized.shuffled/'
folder_original = 'train_data/training-monolingual.tokenized.shuffled/'
preprocessed_folder = '../train_data/training-monolingual.tokenized.shuffled-pre-processed/'
print(preprocessed_folder)
folder_short = 'train_data/preprocessed-short/'
folder_full = 'train_data/preprocessed-full/'

folder_test = 'train_data/heldout/'
# short_folder = 'training-monolingual.tokenized.shuffled_short/'
# words_list_file= 'liste_mots_devoir4.txt'

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self,path,files) -> None:
        self.path = path
        self.files = files

    def __iter__(self):
        for f in self.files:
            for line in open(self.path+f, 'r', encoding="utf8"):
                # assume there's one document per line, tokens separated by whitespace
                yield utils.simple_preprocess(line)

../train_data/training-monolingual.tokenized.shuffled-pre-processed/


In [18]:
def checkpoint(ext,save_model,w2v_model,times,sizes,sent_len):
    
    modelname = f"models/{ext}.w2v"
    csv_times_sizes= f"outputs/{ext}.csv"

    if save_model:
        # stream the model
        #w2v_model.init_sims(replace=True) # deprecated: calls to init_sims() are unnecessary
        w2v_model.save(modelname)
        logging.info(f"saved {modelname}")
 
    np.savetxt(csv_times_sizes, [times,sizes,sent_len],delimiter =", ", fmt ='% s')
    logging.info(f"saved {csv_times_sizes}")
    
    
def train(save_model = True , data = folder,nb_tranches=10,ext='name'):
    # Utilisez gensim pour entrainer des representations vectorielles sur tout ou partie du 1BWC.
    #train by tranche + save the time for each tranche
    print('Starting the training ')
    files = listdir(data)[:nb_tranches]
    sents = []
    times = []
    sizes = []
    sent_len = []
    nb_sentences = 0
    allfiles = False
    if nb_tranches==99:
        allfiles = True
        print('all files')

    training_sample = "training_sample.txt" 
    myfile = open("training_sample.txt", "w")
    myfile.close()

    for i,fn in enumerate(files):
        a = 99 if allfiles else i+1
        print(f"\n{'#-'*10}-Files 1..{a}:")
        logging.info(f"\n{'#-'*10}-Files 1..{a}:")
        t = time()
        
        # concatenate corpus files because LineSentence() takes only a single file
        if not allfiles: 
            os.system("cat "+data+fn+ " >> "+ training_sample)
        t = time()
        
        print(f"- Temps de lecture de sentences (en secondes): {(time() - t)}\n")
        logging.info(f"- Temps de lecture de sentences (en secondes): {(time() - t)}\n")
        l = len(open(data+fn, 'r', encoding="utf8").readlines())
        print(f"Sentences found : {l}")
        logging.info(f"Sentences found : {l}")
        nb_sentences+=l
        print(f"Total Sentences processed : {nb_sentences}")
        logging.info(f"Total Sentences processed : {nb_sentences}")
        sent_len.append(nb_sentences)

    sents = PathLineSentences(data) if allfiles else LineSentence(training_sample)  
    # Time to build model 
    t = time()
    w2v_model = Word2Vec(
        sentences = sents,
        min_count=1, window=5,  vector_size=100,negative=5,
        min_alpha=1e-4, workers=(os.cpu_count()*2 - 1), sample=0.01, alpha=1e-2, epochs=10
        )
    end = round((time() - t),2)
    times.append(end)
    print(f"- Temps d'entrainement (en secondes): {end}\n")

    # Size of the model
    size = round(w2v_model.estimate_memory()['total']/(1024*1024),2)
    sizes.append(size)

    print(f"- Taille du modele sur disque (en octets)): {w2v_model.estimate_memory()}\n\
            Total en MB: {size}")
    logging.info(f"- Taille du modele sur disque (en octets)): {w2v_model.estimate_memory()}\n\
            Total en MB: {size}")

    print("\n- Nombre de mots encodés (= taille du vocab): %d\n" % len(w2v_model.wv.vectors))
    logging.info("\n- Nombre de mots encodés (= taille du vocab): %d\n" % len(w2v_model.wv.vectors))
    #print(w2v_model.most_similar(positive=['abnormalities'], topn = 10))
    #w2v_model.init_sims(replace=True)
    print(w2v_model.wv.most_similar(positive=['abnormalities'])[:10])
    logging.info(w2v_model.wv.most_similar(positive=['abnormalities'])[:10])

    

    checkpoint(ext,save_model,w2v_model,times,sizes,sent_len)

    return times, sent_len, sizes


In [19]:
st = time()
save_model = True 
times, sent_len, sizes = train(save_model,preprocessed_folder, 50,'model')
print(f"Total execution time: {round((time()-st)/60,2)} ")


Starting the training 

#-#-#-#-#-#-#-#-#-#--Files 1..1:
- Temps de lecture de sentences (en secondes): 2.86102294921875e-06

Sentences found : 306068
Total Sentences processed : 306068

#-#-#-#-#-#-#-#-#-#--Files 1..2:
- Temps de lecture de sentences (en secondes): 1.9073486328125e-06

Sentences found : 307000
Total Sentences processed : 613068

#-#-#-#-#-#-#-#-#-#--Files 1..3:
- Temps de lecture de sentences (en secondes): 2.384185791015625e-06

Sentences found : 305915
Total Sentences processed : 918983

#-#-#-#-#-#-#-#-#-#--Files 1..4:
- Temps de lecture de sentences (en secondes): 2.384185791015625e-06

Sentences found : 306362
Total Sentences processed : 1225345

#-#-#-#-#-#-#-#-#-#--Files 1..5:
- Temps de lecture de sentences (en secondes): 2.1457672119140625e-06

Sentences found : 305714
Total Sentences processed : 1531059

#-#-#-#-#-#-#-#-#-#--Files 1..6:
- Temps de lecture de sentences (en secondes): 2.6226043701171875e-06

Sentences found : 305440
Total Sentences processed :

- Temps de lecture de sentences (en secondes): 3.814697265625e-06

Sentences found : 305220
Total Sentences processed : 15302095
- Temps d'entrainement (en secondes): 2725.63

- Taille du modele sur disque (en octets)): {'vocab': 715018000, 'vectors': 572014400, 'syn1neg': 572014400, 'total': 1859046800}
            Total en MB: 1772.93

- Nombre de mots encodés (= taille du vocab): 1430036

[('lesions', 0.8858491778373718), ('tumours', 0.8723723292350769), ('deformities', 0.8477707505226135), ('tumors', 0.8451257348060608), ('mutations', 0.835237979888916), ('cysts', 0.8215368986129761), ('malformations', 0.8200745582580566), ('defects', 0.816528856754303), ('polyps', 0.8129066824913025), ('disorders', 0.8103189468383789)]
Total execution time: 45.94 


/home/kacem/anaconda3/envs/nlp-project-env-conda/lib/python3.8/site-packages/numpy/lib/npyio.py:1378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


model

In [20]:
modelname = f"models/model.w2v"
model = Word2Vec.load(modelname)

In [21]:
vec_king = model.wv['king']
print(vec_king)

[ -0.9015681   -1.4825824    1.0830806   -3.1107342    1.8650055
   1.249914    -1.4759218   -4.1782336   -3.1578116   -1.6441156
  -0.43455067   0.71476966   1.665312    -3.4507391   -3.0525122
   5.592342     1.705296    -0.9882554    0.02499667 -10.047542
   1.6611887   -2.2140872   -0.5830067   -2.7428908   -1.9601579
  -0.9092333   -1.6673864    3.9021957   -1.0805547    2.2152247
   1.3757206    0.40139925   4.455378    -1.4243028    1.1797311
  -0.9060254   -1.0316461   -0.6184769   -0.70201856  -0.8036557
   3.52521     -2.6057773    0.31820267   1.0761127    2.9087982
   0.7554629    0.04572703   2.0367873    1.7707905    1.9300609
   1.0439298    1.6404431   -1.751548    -2.2012062   -0.9391908
   0.6002678    0.78248096   0.78564596  -2.7882662   -7.830961
  -5.0490227    2.6179461    1.478342     1.2702339    2.911088
   2.041558    -0.40612686   0.11329196  -2.429788     5.0052986
   3.054289    -0.26938933   0.09938233  -2.7412643   -1.5820438
  -8.771377     2.2877958   

In [2]:
# Function to convert integer input to vectors for RNN using one hot encoding

def encode(X,seq_len, vocab_size):
    x = np.zeros((len(X),seq_len, vocab_size), dtype=np.float32)
    for ind,batch in enumerate(X):
        for j, elem in enumerate(batch):
            x[ind, j, elem] = 1
    return x

In [3]:
# Generator function to generate infinite-stream of inputs for training

def batch_gen(batch_size=32, seq_len=10, max_no=100):
    # Randomly generate a batch of integer sequences (X) and its sorted counterpart (Y)
    x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)
    y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)

    while True:
	# Generates a batch of input
        X = np.random.randint(max_no, size=(batch_size, seq_len))

        Y = np.sort(X, axis=1)

        for ind,batch in enumerate(X):
            for j, elem in enumerate(batch):
                x[ind, j, elem] = 1

        for ind,batch in enumerate(Y):
            for j, elem in enumerate(batch):
                y[ind, j, elem] = 1

        yield x, y
        x.fill(0.0)
        y.fill(0.0)

In [4]:
def create_model(seq_len, max_no, n_layers, hidden_size):
    model = Sequential()
    # the encoder LSTM
    model.add(LSTM(hidden_size, input_shape=(seq_len, max_no)))
    # in next layer, repeat the input seq_len times
    model.add(RepeatVector(seq_len))
    # decoder RNN, which will return output sequence
    for _ in range(n_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(max_no)))
    model.add(Activation('softmax'))
    return model

In [5]:
# Global parameters.
batch_size=32
seq_len = 15 # number of elements in sequence to sort
max_no = 100 # upper range of the numbers in sequence
hidden_size = 128
n_layers = 2
epochs = 100000

model = create_model(seq_len, max_no, n_layers, hidden_size)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2021-12-15 23:23:10.641355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-15 23:23:10.641482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kacem/usr/lib64
2021-12-15 23:23:10.641551: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kacem/usr/lib64
2021-12-15 23:23:10.641622: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRA

In [6]:
# Now the training loop, we'll sample input batches from the generator function written previously and feed it to the RNN for learning

for ind,(X,Y) in enumerate(batch_gen(batch_size, seq_len, max_no)):
    loss, acc = model.train_on_batch(X, Y)
    # We'll test RNN after each 500 iteration to check how well it is performing
    if ind % 500 == 0:
        testX = np.random.randint(max_no, size=(1, seq_len))
        test = encode(testX, seq_len, max_no)
        #print(testX, ': input sequence')
        print(ind, ': epochs')
        print(acc, ': accuracy')
        loss_str = '{:4.3}'.format(loss)
        acc_str = '{:4.3}'.format(acc)
        
        y = model.predict(test, batch_size=1)
        np_sorted = np.sort(testX)[0]
        rnn_sorted = np.argmax(y, axis=2)[0]
        is_equal = np.array_equal(np_sorted, rnn_sorted)
        print(np_sorted, ': sorted by NumPy algorithm')
        print(rnn_sorted, ': sorted by trained RNN')
        print("\n")
        
        if acc > 0.995 :
            model.save('model.h5')
            break
        
    if ind > epochs:
        # Save trained model
        model.save('model.h5')
        break
    

0 : epochs
0.0062500000931322575 : accuracy
[ 2  3  8 15 21 23 23 46 59 60 60 68 73 84 87] : sorted by NumPy algorithm
[56 56 56 56 56 56 56 56 56 56 46 46 46 46 46] : sorted by trained RNN


500 : epochs
0.10000000149011612 : accuracy
[10 13 15 17 37 38 53 58 60 63 78 82 88 90 94] : sorted by NumPy algorithm
[ 0  8 15 24 31 40 50 58 63 67 73 78 85 96 99] : sorted by trained RNN


1000 : epochs
0.12083332985639572 : accuracy
[ 2  5  8 10 25 28 39 41 47 49 65 73 81 87 88] : sorted by NumPy algorithm
[ 0  5  9 14 20 31 36 38 44 49 66 71 81 86 90] : sorted by trained RNN


1500 : epochs
0.23749999701976776 : accuracy
[ 3 22 24 26 37 44 53 54 55 58 72 80 85 94 94] : sorted by NumPy algorithm
[ 6 20 25 28 35 45 51 53 56 58 71 78 83 93 98] : sorted by trained RNN




KeyboardInterrupt: 